In [1]:
import pandas as pd
import numpy as np

from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath

from sklearn.model_selection import train_test_split

from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional, TimeDistributed, Flatten
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
model = FT_gensim(size=100)
corpus_file = '../task2_data/task2_en_training.tsv'

model.build_vocab(corpus_file=corpus_file)

model.train(
    corpus_file=corpus_file, 
    epochs=model.epochs,
    total_examples=model.corpus_count, 
    total_words=model.corpus_total_words, 
    window = 3)

print(model)

df = pd.read_csv('../task2_data/task2_en_training.tsv', sep='\t')[['class', 'tweet']]

df_ft = df.iloc[np.random.permutation(len(df))]
df_ft['class'] = df_ft['class'].apply(lambda x: int(x))
df_train, df_test = train_test_split(df_ft , test_size=0.2)

FastText(vocab=6047, size=100, alpha=0.025)


In [3]:
#Split into training and test data.
X_train = df_train['tweet']
X_test = df_test['tweet']
y_train = df_train['class']
y_test = df_test['class']

# it's special. we can't work with other format
X_train = np.array(X_train)
X_test = np.array(X_test)

#по сути на этом этапе можно слегка почистить составляющие твитов
for i in range(len(X_train)):
    sentence = X_train[i].split(' ')
    sentence = list(map(lambda x: model[x], sentence))
    X_train[i] = sentence

for i in range(len(X_test)):
    sentence = X_test[i].split(' ')
    sentence = list(map(lambda x: model[x], sentence))
    X_test[i] = sentence

maxima = 0
for ar in X_train:
    if len(ar)>maxima: maxima = len(ar)
for ar in X_test:
    if len(ar)>maxima: maxima = len(ar)
max_len = maxima + 1
X_train.shape

/home/alexkay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/alexkay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(16435,)

In [4]:
def add_pads(ars, max_len):
    for i in range(len(ars)):
        while len(ars[i]) < max_len: ars[i].insert(0,np.array([0 for i in range(100)]))
    return ars
X_train = add_pads(X_train, max_len)
X_test = add_pads(X_test, max_len)
X_train = X_train.tolist()

In [5]:
X_train = np.array(X_train).reshape(len(X_train), len(X_train[1]), 100)

In [6]:
# configure network
#n_batch = len(X) # !!!!!
n_batch = 5
n_epoch = 5
n_neurons = 10

# design network
model = Sequential()
model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(2, activation='softmax'))

model.compile(loss='binary_crossentropy', 
              optimizer='rmsprop', 
              metrics=['acc'])
print(model.summary())

# fit network
model.fit(X_train, to_categorical(y_train, num_classes=2, dtype='float32'), epochs=n_epoch, batch_size=n_batch)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (5, 10)                   4440      
_________________________________________________________________
dense_1 (Dense)              (5, 2)                    22        
Total params: 4,462
Trainable params: 4,462
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
16435/16435 [==============================] - 30s 2ms/step - loss: 0.2933 - acc: 0.9064
Epoch 2/5
16435/16435 [==============================] - 30s 2ms/step - loss: 0.2798 - acc: 0.9066
Epoch 3/5
16435/16435 [==============================] - 30s 2ms/step - loss: 0.2752 - acc: 0.9067
Epoch 4/5
16435/16435 [==============================] - 30s 2ms/step - loss: 0.2713 - acc: 0.9063
Epoch 5/5
16435/16435 [==============================] - 30s 2ms/step - loss: 0.2705 - acc: 0.9071


In [8]:
X_test = X_test.tolist()
X_test = np.array(X_test).reshape(len(X_test), len(X_test[1]), 100)
X_test.shape, X_train.shape, 

((4109, 63, 100), (16435, 63, 100))

In [9]:
n_batch = 1
# re-define model
new_model = Sequential()
new_model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X_test.shape[1], X_test.shape[2]), stateful=True))
new_model.add(Dense(2, activation='softmax'))
# copy weights
old_weights = model.get_weights()
new_model.set_weights(old_weights)
# compile model
new_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [10]:
scores = new_model.evaluate(X_test, 
                            to_categorical(y_test, num_classes=2, dtype='float32'), 
                            batch_size=1, 
                            verbose=0)

In [11]:
print("Model Accuracy: %.2f%%" % (scores[1]*100))

Model Accuracy: 90.44%
